# Analyze Cycle 7 reports for County Executive candidates currently in office

In [1]:
import pandas as pd
from PyPDF2 import PdfReader
import re

In [2]:
CONTRIB_REGEX = r'Full Name of Contrib.*\n(.*)\n(.*)\n(.*)\n(.*)\n(.*)\n(.*)\n(\d+)\n(\d+)\n(\d+)\n'
RECEIPT_REGEX = r'Full Name \n(.*)\n(.*)\n(.*)\n(.*)\n(.*)\n(.*)\n(\d+)\n(\d+)\n(\d+)\nReceipt Description\n.*\n'
UNITEMIZED_REGEX = r'Unitemized  Contributions Received - \$ 50.00 or Less Per Contributor\nTOTAL for the Reporting Period           \(1\)\n\$\n(.*)\n'

def flatten(l):
    return [item for sublist in l for item in sublist]

def get_contributions(filename):
    contributions = process_report(filename)

    df = pd.DataFrame(
        flatten(contributions),
        columns=['name', 'address1', 'city', 'state', 'zipcode', 'amount', 'month', 'day', 'year'])
    df['name'] = df.name.str.upper()
    df['amount'] = pd.to_numeric(df.amount.str.replace(',', ''))

    return df

def process_report(filename):
    reader = PdfReader(filename)
    text = ""

    for page in reader.pages:
        text += page.extract_text() + "\n"

    unitemized = re.findall(UNITEMIZED_REGEX, text)
    if len(unitemized) == 0:
        unitemized = []
    else:
        unitemized = [(
            'UNITEMIZED',
            '',
            'PITTSBURGH',
            'PA',
            '',
            unitemized[0],
            '',
            '',
            ''
        )]

    return [
        re.findall(CONTRIB_REGEX, text),
        re.findall(RECEIPT_REGEX, text),
        unitemized
    ]

### Load analysis from previous reports

In [3]:
pastleaders = pd.read_csv('output/wein-inna-lamb-top.csv')

### Load Cycle 7 reports for 2022

##### John Weinstein (state committee)

In [4]:
df_weinstein = get_contributions('input/blank.pdf')
df_weinstein = df_weinstein.rename(columns={'amount': 'amount_wein'})

df_weinstein['name'] = df_weinstein.name.replace({
    "AMALGAMATED TRANSIT UNION COPE": "ATU COPE VOLUNTARY ACCOUNT",
    "IBEW LOCAL UNION NO. 5 PAC": "LOCAL 0005 IBEW PAC",
    "MIDATLANTIC POLITICAL LEAGUE - MALPA": "MID-ATLANTIC LABORERS' POLITICAL LEAGUE",
    "STEAMFITTERS LOCAL UNON #449": "LOCAL 0449 STEAMFITTERS UNION PAC",
    "STEAMFITTERS LOCAL UNION 449 PAC FUND": "LOCAL 0449 STEAMFITTERS UNION PAC",
    "PLUMBERS LOCAL UNION NO. 27 PAC": "LOCAL 0027 PLUMBERS UNION PAC",
    "BRICKLAYERS &AMP; ALLIED CRAFTWORKERS LOCAL 9 PAC": "LOCAL 0009 BRICKLAYERS & ALLIED CRAFTWORKERS PA PAC",
    "PITTSBURGH FIRE FIGHTERS LOCAL NO 1 FIRE PAC ACCOUNT": "PGH FIRE FIGHTERS LOCAL #1 FIRE PAC",
    "TEAMSTERS LOCAL UNION 249 - DRIVE FUND": "LOCAL 0249 TEAMSTERS DRIVE",
    "U.W.U.A. LOCAL 433 PAC": "LOCAL 0433 UWUA (UTILITY WORKERS)",
    "TEAMSTER JOINT COUNCIL 40 PAC": "TEAMSTERS JT COUNCIL 40 PAC",
    "AFSCME COUNCIL 13 POLITICAL &AMP; LEGISLATIVE": "AFSCME COUNCIL 13 POL & LEG ACCT",
    # Laborers
    "LABORERS DISTRICT COUNCIL OF WESTERN PENNSYLVANIA": "WESTERN PENNSYLVANIA LABORERS 2019 PAC",
    "LABORERS' DISTRICT COUNCIL OF WESTERN PENNSYLVANIA": "WESTERN PENNSYLVANIA LABORERS 2019 PAC",
    "WESTERN PA LABORERS UNION PAC": "WESTERN PENNSYLVANIA LABORERS 2019 PAC",
    "WESTERN PENNSYLVANIA LABORERS": "WESTERN PENNSYLVANIA LABORERS 2019 PAC",
    "WESTERN PENNSYLVANIA LABORERS' PAC": "WESTERN PENNSYLVANIA LABORERS 2019 PAC",
})

In [5]:
df_weinstein_topcontribs = df_weinstein.groupby('name').amount_wein.sum().to_frame().reset_index()

weinstein_sum = df_weinstein.amount_wein.sum()
df_weinstein_topcontribs['pct_wein'] = df_weinstein_topcontribs.amount_wein / weinstein_sum

df_weinstein_topcontribs.sort_values(by='amount_wein', ascending=False).head(10)

,name,amount_wein,pct_wein


##### Sara Innamorato (state committee)

In [6]:
df_innamorato = get_contributions('input/blank.pdf')
df_innamorato = df_innamorato.rename(columns={'amount': 'amount_inna'})

df_innamorato['name'] = df_innamorato.name.replace({
    "1776 PAC": "1776 PAC (UFCW)",
    "AFSCME COUNCIL 13": "AFSCME COUNCIL 13 POL & LEG ACCT",
    "EVAN SEGAL": "EVAN J. SEGAL",
    "PENNSYLVANIA SEIU COPE": "SEIU HEALTHCARE PA COPE",
    "PFT POLITICAL ACTION FUND": "PFT POL ACTION FUND (PGH FED TEACHERS)",
    "TEAMSTERS LOCAL UNION 249": "LOCAL 0249 TEAMSTERS DRIVE",
    "TEAMSERS LOCAL UNION 249": "LOCAL 0249 TEAMSTERS DRIVE",
    "TEAMSTERS LOCAL UNION 249 DRIVE FUND": "LOCAL 0249 TEAMSTERS DRIVE",
    "WESTERN PENNSYLVANIA LABORERS POLITICAL ACTION FUND": "WESTERN PENNSYLVANIA LABORERS 2019 PAC",
})

In [7]:
df_innamorato_topcontribs = df_innamorato.groupby('name').amount_inna.sum().to_frame().reset_index()

innamorato_sum = df_innamorato.amount_inna.sum()
df_innamorato_topcontribs['pct_inna'] = df_innamorato_topcontribs.amount_inna / innamorato_sum

df_innamorato_topcontribs.sort_values(by='amount_inna', ascending=False).head(10)

,name,amount_inna,pct_inna


##### Michael Lamb (local committee -- use hand-rolled CSV)

In [8]:
df_lamb = pd.read_csv('input/Lamb_2022_C7.csv')
df_lamb = df_lamb[df_lamb.amount_lamb > 0]

In [9]:
df_lamb_topcontribs = df_lamb.groupby('name').amount_lamb.sum().to_frame().reset_index()

lamb_sum = df_lamb.amount_lamb.sum()
df_lamb_topcontribs['pct_lamb'] = df_lamb_topcontribs.amount_lamb / lamb_sum

df_lamb_topcontribs.sort_values(by='amount_lamb', ascending=False).head(10)

,name,amount_lamb,pct_lamb


### Compare Cycle 7 to previous reports

In [10]:
pd.merge(
    df_weinstein_topcontribs,
    pastleaders[['name', 'amount_wein', 'pct_wein']],
    on='name', how='outer',
).sort_values(by='pct_wein_y', ascending=False).head(10)

,amount_wein_x,pct_wein_x,name,amount_wein_y,pct_wein_y
321,NaN,NaN,WESTERN PENNSYLVANIA LABORERS 2019 PAC,90000.0,0.180471
22,NaN,NaN,ATU COPE VOLUNTARY ACCOUNT,37614.0,0.075425
103,NaN,NaN,FNB CORPORATION PAC,22500.0,0.045118
241,NaN,NaN,MID-ATLANTIC LABORERS' POLITICAL LEAGUE,22000.0,0.044115
113,NaN,NaN,FRIENDS OF RANDY MARTINI,20000.0,0.040105
203,NaN,NaN,LOCAL 0449 STEAMFITTERS UNION PAC,15000.0,0.030078
198,NaN,NaN,LOCAL 0005 IBEW PAC,10000.0,0.020052
10,NaN,NaN,AMALGAMATED TRANSIT UNION - LOCAL 85,10000.0,0.020052
146,NaN,NaN,JAMES P. GRANT,10000.0,0.020052
41,NaN,NaN,CHARLES HAMMEL III,7500.0,0.015039


In [11]:
pd.merge(
    df_lamb_topcontribs,
    pastleaders[['name', 'amount_lamb', 'pct_lamb']],
    on='name', how='outer',
).sort_values(by='pct_lamb_y', ascending=False).head(10)

,amount_lamb_x,pct_lamb_x,name,amount_lamb_y,pct_lamb_y
198,NaN,NaN,LOCAL 0005 IBEW PAC,56250.0,0.142927
200,NaN,NaN,LOCAL 0027 PLUMBERS UNION PAC,48550.0,0.123362
203,NaN,NaN,LOCAL 0449 STEAMFITTERS UNION PAC,40350.0,0.102526
1004,NaN,NaN,LOCAL 0066 PAC CLUB,37000.0,0.094014
994,NaN,NaN,LOCAL 0690 PLUMBERS UNION POL ACTION FUND,27500.0,0.069875
999,NaN,NaN,LOCAL 0542 IUOE OPER ENG (NORRISTN),25000.0,0.063523
552,NaN,NaN,GREATER PA CARPENTERS PAC,18050.0,0.045864
1002,NaN,NaN,INTL UNION PAINTERS ALLIED TRADE,14000.0,0.035573
321,NaN,NaN,WESTERN PENNSYLVANIA LABORERS 2019 PAC,10000.0,0.025409
1010,NaN,NaN,INTL BROTHERHOOD BOILERMAKERS 154,7100.0,0.018041


In [12]:
pd.merge(
    df_innamorato_topcontribs,
    pastleaders[['name', 'amount_inna', 'pct_inna']],
    on='name', how='outer',
).sort_values(by='pct_inna_y', ascending=False).head(10)

,amount_inna_x,pct_inna_x,name,amount_inna_y,pct_inna_y
315,NaN,NaN,UNITEMIZED,35293.34,0.109579
530,NaN,NaN,EVAN J. SEGAL,16020.63,0.049741
912,NaN,NaN,SEIU HEALTHCARE PA COPE,11250.00,0.034929
841,NaN,NaN,PENNSYLVANIA SIERRA CLUB PAC,10000.00,0.031048
478,NaN,NaN,DAVID TURNER,7650.00,0.023752
840,NaN,NaN,PENNSYLVANIA DEMOCRATIC PARTY,7613.90,0.023640
970,NaN,NaN,WOMEN FOR THE FUTURE,6500.00,0.020181
796,NaN,NaN,NANCY BERNSTEIN,6500.00,0.020181
342,NaN,NaN,ALLEGHENY COUNTY DEMOCRATIC DELEGATION,5750.00,0.017853
872,NaN,NaN,REPRESENT PAC,5500.00,0.017076
